# Лабораторная работа 3. Модель однофазной одноканальной замкнутой системы обслуживания

## Вариант 2

##### Выполнил: Борисочкин М. И. РТ5-81Б

### Задание 1

In [ ]:
# Генерация начальных значений 
Variant <- 2
set.seed(Variant) 
k <- sample(c(4:9), 1)
pp1 <- runif(4)
pp2 <- runif(3)
pp3 <- runif(2)
p1 <- pp1/sum(pp1)
p2 <- c(c(0), pp2/sum(pp2))
p3 <- c(c(0,0) ,pp3/sum(pp3))
p4 <- c(0,0,0,1)
P <- data.frame()
P <- rbind(P, p1)
P <- rbind(P, p2)
P <- rbind(P, p3)
P <- rbind(P, p4)
rownames(P)<-c("p1", "p2", "p3", "p4")
colnames(P)<-c("", "", "", "")
View(P)
print(paste("k =", as.character(k)))

,,,,
,<dbl>,<dbl>,<dbl>,<dbl>
p1,0.2941768,0.2401275,0.07038554,0.3953102
p2,0.0000000,0.4949811,0.06776147,0.4372574
p3,0.0000000,0.0000000,0.45974212,0.5402579
p4,0.0000000,0.0000000,0.00000000,1.0000000


[1] "k = 8"


#### Теоретическое решение



Граф состояний:

![image](https://raw.githubusercontent.com/RDcool/BMSTU_AHP/main/Lab%203/States%20graph.svg)

In [ ]:
# Создание начальных матриц

# Вероятность начального состояния
p0 <- matrix(c(1, 0, 0, 0), ncol = 4, nrow = 1, byrow = TRUE)
Pmatrix <- data.matrix(P)

In [ ]:
# Установка и подключение библиотеки matrixcalc
install.packages("matrixcalc")
library(matrixcalc)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Функция расчёта вероятностей состояний на шаге k
calculateprobs <- function(K)
{
  res <- data.frame(p0 %*% matrix.power(Pmatrix, K))

  rownames(res) <- "Вероятность"
  colnames(res) <- c("S1", "S2", "S3", "S4")

  return(res)
}

In [ ]:
# Рассчитанные вероятности состояний на шаге k
calculateprobs(k)

,S1,S2,S3,S4
,<dbl>,<dbl>,<dbl>,<dbl>
Вероятность,5.608765e-05,0.004241942,0.003571874,0.9921301


In [ ]:
# Рассчитанные вероятности состояний на шаге k - 1
calculateprobs(k - 1)

,S1,S2,S3,S4
,<dbl>,<dbl>,<dbl>,<dbl>
Вероятность,0.0001906597,0.008477413,0.006490623,0.9848413


In [ ]:
# Рассчитанные вероятности состояний на шаге k - 2
calculateprobs(k - 2)

,S1,S2,S3,S4
,<dbl>,<dbl>,<dbl>,<dbl>
Вероятность,0.0006481125,0.01681232,0.01154077,0.9709988


#### Численное решение

In [ ]:
# Функция перехода из состояния в состояние
makestep <- function(state, prob)
{
  curstate <- state
  rowprob <- P[state,]

  if(prob <= rowprob[1]) curstate <- 1
  else if(prob > rowprob[1] & prob <= rowprob[1] + rowprob[2]) curstate <- 2
  else if(prob > rowprob[1] + rowprob[2] & prob <= rowprob[1] + rowprob[2] + rowprob[3]) curstate <- 3
  else if(prob > rowprob[1] + rowprob[2] + rowprob[3] & prob <= 1) curstate <- 4

  return(curstate)
}

In [ ]:
# Функция проведения одиночного эксперимента
conductexperiment <- function(P, K)
{
  resstates <- c()
  curstate <- 1 # Начальное состояние

  for(i in 1:K)
  {
    prob <- runif(1)
    curstate <- makestep(curstate, prob)
    resstates <- append(resstates, curstate)
  }
  return(resstates)
}

In [ ]:
# Проведение экпериментов
resdata <- data.frame()
N <- 150000 # Число экспериментов

for (i in 1:N)
{
  resvector <- c(1)
  res <- conductexperiment(P, k)
  resvector <- append(resvector, res)
  resdata <- rbind(resdata, resvector)
}

colnames(resdata) <- c(0:8)
rownames(resdata) <- c(1:N)

In [ ]:
# Функция подсчёта вероятности состояний на k-том шаге
getstateprobs <- function(K)
{
  probsvect <- c(0, 0, 0, 0)
  for (i in 1:4) probsvect[i] <- sum(resdata[,K] == i)/length(resdata[,K])
  
  probsdf <- data.frame(probsvect)
  probsdf <- t(probsdf)

  rownames(probsdf) <- "Вероятность"
  colnames(probsdf) <- c("S1", "S2", "S3", "S4")

  return(probsdf)
}

In [ ]:
# Вероятности состояний на шаге k
getstateprobs(k)

,S1,S2,S3,S4
Вероятность,0.0001533333,0.008693333,0.006806667,0.9843467


In [ ]:
# Вероятности состояний на шаге k - 1
getstateprobs(k - 1)

,S1,S2,S3,S4
Вероятность,0.00058,0.01752667,0.01168,0.9702133


In [ ]:
# Вероятности состояний на шаге k - 2
getstateprobs(k - 2)

,S1,S2,S3,S4
Вероятность,0.002133333,0.03362,0.01946,0.9447867


### Задание 2

In [ ]:
# Генерация начальных условий
Variant <- 2
set.seed(Variant) 
k <- sample(c(10:25), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:5), 1)
View(data.frame(k, t1, t2))

k,t1,t2
<int>,<int>,<int>
14,20,3


In [ ]:
# Вычисление интенсивностей входа и выхода, а также нагрузки системы
lambda <- k/t1
mu <- 1/t2
y <- lambda/mu

View(data.frame(lambda, mu, y))

lambda,mu,y
<dbl>,<dbl>,<dbl>
0.7,0.3333333,2.1


Выше видно, что серврер перегружен (y > 0) => задача не разрешима. Поэтому количество программистов (k) было снижено до 6.

In [ ]:
k <- 6
  
lambda <- k/t1
y <- lambda/mu

View(data.frame(lambda, mu, y))

lambda,mu,y
<dbl>,<dbl>,<dbl>
0.3,0.3333333,0.9


#### Теоритическая часть

In [ ]:
# Вероятность, что сервер будет свободен
P0 <- 1 - y

In [ ]:
# Вероятность того, что программа не будет выполнена сразу же,
# как только она поступила на терминал
P <- 1 - P0
P

[1] 0.9

In [ ]:
# Cреднее время до получения пользователем результатов реализации
Wq <- y^2/(lambda*(1 - y))
Wq

[1] 27

In [ ]:
# Среднее количество программ, ожидающих выполнения на сервере
Lq <- lambda * Wq
Lq

[1] 8.1

#### Эксперементальная часть

Для проведения эксперимента библиотеку для дискретно-событийного моделирования [simmer](https://r-simmer.org/index.html)

In [ ]:
# Установка библиотеки
install.packages("simmer", dependencies = TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("simmer", dependencies = TRUE):
“installation of package ‘simmer’ had non-zero exit status”


In [ ]:
# Подключение библиотеки
library("simmer")

In [ ]:
set.seed(Variant)

# Создание траектории программ
m.queue <- trajectory() %>%
  seize("server", amount=1) %>% # Занятие сервера
  timeout(function() rexp(1, mu)) %>% # Обслуживание программы
  release("server", amount=1) # Освобождение сервера

# Создание окружения симуляции
mm.env <- simmer() %>%
  add_resource("server", capacity=1, queue_size=Inf) %>% # Создание ресурса "Сервер"
  add_generator("program", m.queue, function() rexp(1, lambda)) %>% # Создание потока программ
  run(until=10250)

# Проведение 7500 экспериментов
mm.envs <- lapply(1:5000, function(i) {mm.env})

In [ ]:
# Вероятность того, что программа не будет выполнена сразу же,
# как только она поступила на терминал
serverData <- get_mon_resources(mm.envs)
sum(serverData$server == 1)/length(serverData$server)

[1] 0.9520415

In [ ]:
# Cреднее время до получения пользователем результатов реализации
programsData <- get_mon_arrivals(mm.envs)
mean(programsData$end_time - programsData$start_time)

[1] 28.35902

In [ ]:
# Среднее количество программ, ожидающих выполнения на сервере
mean(serverData$queue)

[1] 8.059948

### Дополнительное задание

In [ ]:
# Генерация начальных условий
Variant <- 2
set.seed(Variant) 
pp <- runif(3)
p1 <- pp[1]/sum(pp)
p2 <- pp[2]/sum(pp)
p3 <- pp[3]/sum(pp)
k <- sample(c(4:8), 1)
m <- sample(c(1:k), 1)
View(data.frame(p1, p2, p3, k, m))

p1,p2,p3,k,m
<dbl>,<dbl>,<dbl>,<int>,<int>
0.1265812,0.4808862,0.3925326,4,1


#### Теоритический метод

Для расчёта веровятности рассмотрим следуюшие варианты пребывания точки на расстоянии не более единицы за четыре шага:

- Точка 4 раза стояла
- Точка 3 раза стояла и сделала шаг
- Точка 2 раза стояла, сделала шаг влево и шаг вправо
- Точка стояла 1 раз, сделала 2 шага в одну сторону и один в противоположную
- Точка сделала два шага влево и два шага вправо

Действия описанные в каждом пункте, кроме первого, можно выполнять в разном порядке, что мы учтём с помощью формулы перестановки с повторениями:

$\overline{P}(k_1, k_2, k_3) = \frac{(k_1 + k_2 + k_3)!}{k_1! ⋅ k_2! ⋅ k_3!}$

In [ ]:
# Вероятность, что точка после k шагов окажется на расстоянии не более m
p1^4 + 4*p1^3*p2 + 4*p1^3*p3 + 6*p2^2*p3^2 + 12*p2^2*p3*p1 + 12*p3^2*p2*p1 + 12*p1^2*p2*p3

[1] 0.5078594

#### Численный метод

In [ ]:
# Функция проведения одиночного эксперимента
conductexperiment <- function(K)
{
  resstates <- c()
  curstate <- 0 # Начальное состояние

  for(i in 1:K)
  {
    prob <- runif(1)

    if(prob <= p1) curstate <- curstate
    else if(prob > p1 & prob <= p1 + p2) curstate <- curstate + 1
    else if(prob > p1 + p2 & prob <= 1) curstate <- curstate - 1

    resstates <- append(resstates, curstate)
  }
  return(resstates)
}

In [ ]:
# Проведение экпериментов
resdata <- data.frame()
N <- 100000 # Число экспериментов

for (i in 1:N)
{
  resvector <- c(0)
  res <- conductexperiment(k)
  resvector <- append(resvector, res)
  resdata <- rbind(resdata, resvector)
}

colnames(resdata) <- c(0:4)
rownames(resdata) <- c(1:N)

In [ ]:
# Вероятность, что точка после k шагов окажется на расстоянии не более m
sum(abs(resdata$"4") <= m)/length(resdata$"4")

[1] 0.50664